In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [4]:
from langchain.indexes import VectorstoreIndexCreator

In [8]:
# !pip install docarray

In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [10]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [11]:
response = index.query(query)

In [17]:
Markdown(response)



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

In [18]:
loader = CSVLoader(file_path=file)

In [19]:
docs = loader.load()

In [42]:
Markdown(str(docs[0]))

page_content=': 715\nname: Classic Comfort Check Shirt, Traditional Fit\ndescription: Our check-patterned button-down shirt looks as crisp at the end of the day as it does at the beginning. Size & Fit: Traditional Fit: Relaxed through the chest, sleeve and waist. Fabric & Care: 100% 80s two-ply cotton. Features wrinkle-free performance that never washes out. Our innovative TrueCool® fabric wicks moisture away from your skin and helps it dry quickly. Machine wash and dry. Additional Features: Button-down collar. Single patch pocket. Imported.' metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 715}

In [21]:
Markdown('tetst\n\nfdsfgsdf')

tetst

fdsfgsdf

In [22]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [23]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [24]:
print(embed[:5])

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [25]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [26]:
query = "Please suggest a shirt like my granpa wears"

In [27]:
docs = db.similarity_search(query)

In [28]:
len(docs)

4

In [29]:
docs[0]

Document(page_content=': 715\nname: Classic Comfort Check Shirt, Traditional Fit\ndescription: Our check-patterned button-down shirt looks as crisp at the end of the day as it does at the beginning. Size & Fit: Traditional Fit: Relaxed through the chest, sleeve and waist. Fabric & Care: 100% 80s two-ply cotton. Features wrinkle-free performance that never washes out. Our innovative TrueCool® fabric wicks moisture away from your skin and helps it dry quickly. Machine wash and dry. Additional Features: Button-down collar. Single patch pocket. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 715})

In [30]:
retriever = db.as_retriever()

In [31]:
llm = ChatOpenAI(temperature = 0.0)


In [32]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [33]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [34]:
display(Markdown(response))

| Shirt Name | Sun Protection | Summary |
| --- | --- | --- |
| Classic Comfort Check Shirt, Traditional Fit | No | Wrinkle-free performance shirt made of 100% 80s two-ply cotton with TrueCool® fabric that wicks moisture away from the skin. Features a button-down collar and a single patch pocket. |
| Classic Plaid Short-Sleeve Shirt | No | Lightweight and breathable linen shirt made from pure European flax. Features a single patch pocket and a shirttail hem that can be tucked in or worn out. |
| Smooth Comfort Check Shirt, Slightly Fitted | Yes | Slightly fitted check shirt made of 100% cotton poplin with wrinkle-free performance and TrueCool® fabric that wicks moisture away from the skin. Features a button-down collar and a single patch pocket. |
| Timeless 1933 Plaid Shirt, Slim Fit | No | Slim fit chamois shirt made of 100% cotton flannel woven in Portugal. Brushed and garment-washed for a vintage broken-in feel. Features angled flap pockets with custom buttons, a spread collar, and a shirttail hem. Can be worn as an over-shirt or a casually untucked layer. |

In [35]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [36]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [37]:
response = qa_stuff.run(query)



> Entering new  chain...

> Finished chain.


In [38]:
display(Markdown(response))

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is machine washable and dryable. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 255 | Sun Shield Shirt | This shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is handwashable and line dry. It is rated UPF 50+ for superior protection from the sun's UV rays. It is abrasion-resistant and wicks moisture for quick-drying comfort. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are all designed to be lightweight and comfortable in hot weather. They all have front and back cape venting that lets in cool breezes and two front bellows pockets. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon. The Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber and is abrasion-resistant and wicks moisture for quick-drying comfort.

In [39]:
response = index.query(query, llm=llm)

In [40]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])